## Interaction network template
Produces an interaction list from a ```.mymridon``` experiment file and saves it as a csv, which can be further analyzed in the optional part or exported into other software such as RStudio.  
This notebook is a walk trough a sample usage of the following:
* the py-myrmidon library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* the facetnet library ([Documentation](https://c4science.ch/source/facet_unil/))


In [1]:
import py_fort_myrmidon as fm
import pandas as pd
import numpy as np  # Fundamental math library in python. Here used only for convience: to save the csv.
from datetime import datetime, timedelta  # For convenient handling of time and date
import networkx as nx  # Optional: for general graph analysis and plotting
import csv
import facetnet  # Optional: for community analysis
import matplotlib.pyplot as plt  # Optional: for plorring
# Optional: makes plots interactive:
%matplotlib widget

In [3]:
f_myrmidon = "/media/work/common_data/SNG/SNG_1_enlarged.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
t_start = datetime(2021, 11, 2, 0, 1).astimezone(tz=None)  # <year, month, day, hour, minute>
t_end = datetime(2021, 11, 8, 23, 59).astimezone(tz=None)

The following is an iterator for fort mymrmidon time over days in a period. See Ant metadata template for explanation.

In [4]:
def fm_time_range(start_datetime, end_datetime):
    for n in range(int((end_datetime - start_datetime).days) + 1):
        yield fm.Time(start_datetime + timedelta(n))

A matcher is used to filter interactions of a certain type ("body part 1 with body part 1") that were recorded during a user define period from ```t_start``` to ```t_end```. The ```for``` loop takes the ```ant_id``` of both individuals invovled in the interaction and updates the egde weight in the count graph, as well as in the interaction duration graph. The two edge lists are then saved to csv with name that consists of the experiment name and the dates. This is to avoid confusion, any string can serve as a file name.

In [5]:
m = fm.Matcher.InteractionType(2, 2) # At the moment CHECK EVERY TIME

I_interaction = []
G_counts = nx.Graph()
G_counts.add_nodes_from(exp.Ants.keys())
G_seconds = nx.Graph()
G_seconds.add_nodes_from(exp.Ants.keys())

for t_begin in fm_time_range(t_start, t_end):
    interactions = fm.Query.ComputeAntInteractions(exp,
                                                   start=t_begin,
                                                   end=t_begin.Add(fm.Duration.Parse('24h')),
                                                   matcher=m)
    for ia in interactions[1]:
        I_interaction.append(list(ia.IDs) + [ia.Start, ia.End, ia.Space] + [any((ia.Types[:]==[1,1]).all(1)),
                                                                           any((ia.Types[:]==[1,2]).all(1)),
                                                                           any((ia.Types[:]==[2,1]).all(1)),
                                                                           any((ia.Types[:]==[2,2]).all(1))])

with open("interaction_list_{}_{}_{}.csv".format(exp.Name, t_start, t_end),'w') as f:
    write=csv.writer(f)
    write.writerow(['id1','id2','start','end','space','1-1','1-2','2-1','2-2'])
    write.writerows(I_interaction)
    